# Blob-level Analysis

In [13]:
x=list()
classes=c("integer","character","numeric","integer","integer")
for(i in 0:2){
    path=paste("../data/blobs/bsample.sltcd.",i,"y",sep="")
    x[[i+1]]=read.table(path,sep=";",colClasses=classes,na.strings=c("null"))
}
names(x)=c("All","1y","2y")

In [23]:
for(i in 1:3){
    names(x[[i]])=c("s","l","t","c","d")
    x[[i]]$c=as.logical(x[[i]]$c)
    x[[i]]$b=is.na(x[[i]]$s)
}    

In [24]:
tmp=list()
for(i in 1:3) {
    tmp[[i]]=names(table(x[[i]]$l)[table(x[[i]]$l)>100000])
}
tmp2=intersect(intersect(tmp[[1]],tmp[[2]]),tmp[[3]])
for(i in 1:3) {
    x[[i]]$ll=as.factor(ifelse(((x[[i]]$l %in% tmp2) & (x[[i]]$l != "null") & (x[[i]]$l != "Unknown")),
                     as.character(x[[i]]$l), "Other"))
}
ty=names(table(x[[i]]$ll))

## Variables

- s: Size 
- l: Language
- t: Time 
- c: Is copied
- d: Downstream projects
- b: Is binary

In [26]:
for(i in 1:3) {
    print(names(x)[i])
    print(summary(x[[i]]))
}

[1] "All"
       s                  l                   t                 c            
 Min.   :        2   Length:122668368   Min.   :0.000e+00   Mode :logical    
 1st Qu.:      553   Class :character   1st Qu.:1.489e+09   FALSE:114191192  
 Median :     1358   Mode  :character   Median :1.553e+09   TRUE :8477176    
 Mean   :    16637                      Mean   :1.531e+09                    
 3rd Qu.:     4144                      3rd Qu.:1.597e+09                    
 Max.   :613836533                      Max.   :8.951e+09                    
 NA's   :27398022                                                            
       d                b                   ll          
 Min.   :     0.0   Mode :logical    Other   :76340563  
 1st Qu.:     0.0   FALSE:95270346   JS      :18162569  
 Median :     0.0   TRUE :27398022   java    : 5727418  
 Mean   :     1.5                    C       : 4126564  
 3rd Qu.:     0.0                    Markdown: 3887356  
 Max.   :863568.0       

### 1. What fraction of blobs are copied?

In [27]:
for(i in 1:3) {
    print(c(names(x)[i],sum(x[[i]]$c)/dim(x[[i]])[1]))
    print(c("Binaries excluded:",sum(x[[i]][!x[[i]]$b,"c"])/dim(x[[i]][!x[[i]]$b,])[1]))
}

[1] "All"                "0.0691064545669997"
[1] "Binaries excluded:" "0.0633983002433937"
[1] "1y"                 "0.0614612341053134"
[1] "Binaries excluded:" "0.0568222332892975"
[1] "2y"                 "0.0758498980189842"
[1] "Binaries excluded:" "0.0703809751872513"


### 2. Does the propensity to copy vary with language?

In [28]:
for(i in 2:3){
    print(names(x)[i])
    test=aov(c ~ ll, data=x[[i]])
    print(summary(test));
}

[1] "1y"
                   Df  Sum Sq Mean Sq F value Pr(>F)    
ll                 21   32024  1524.9   26569 <2e-16 ***
Residuals   111460968 6397464     0.1                   
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
[1] "2y"
                  Df  Sum Sq Mean Sq F value Pr(>F)    
ll                21   33032  1572.9   22565 <2e-16 ***
Residuals   84636809 5899730     0.1                   
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


#### 2.1. Which languages are different?

In [29]:
sv=0.005
for(i in 2:3){
    print(names(x)[i])
    for(l in ty){ 
        test=t.test(x[[i]][x[[i]]$ll==l,"c"],x[[i]][x[[i]]$ll!=l,"c"])
        if(test$p.value<sv){
            mean=sum(x[[i]][x[[i]]$ll==l,"c"])/dim(x[[i]][x[[i]]$ll==l,])[1]
            print(c(l,mean,sprintf("t: %f",test$statistic)))
        }else{
            print(c(l,"Not significant!"))
        }
    }
}

[1] "1y"
[1] "C"                "0.13247575671542" "t: 427.762915"   
[1] "CSS"                "0.0600209492330253" "t: -6.313934"      
[1] "Cs"                 "0.0535597530852168" "t: -50.682979"     
[1] "Fml"                "0.0665055831847612" "t: 8.366666"       
[1] "Go"                 "0.0676721571740425" "t: 16.688648"      
[1] "JS"                 "0.0454613563374903" "t: -323.544157"    
[1] "Kotlin"            "0.025943061789138" "t: -122.991248"   
[1] "Markdown"           "0.0319865798047435" "t: -316.557567"    
[1] "ObjectiveC"         "0.0727411039821064" "t: 24.990494"      
[1] "Other"              "0.0623895418770649" "t: 52.436575"      
[1] "PY"                 "0.0544774489144131" "t: -48.208890"     
[1] "R"                  "0.0855873728624931" "t: 39.251335"      
[1] "Rust"               "0.0556607054750754" "t: -12.593480"     
[1] "Scala"              "0.0306845674939291" "t: -84.235546"     
[1] "Sql"                "0.0573553387472549" "t: -7.792719"  

In [73]:
for(i in 2:3) {
    print(names(x)[i])
    glm=glm(c~ll, data=x[[i]], family=binomial)
    print(summary(glm))
}

[1] "1y"

Call:
glm(formula = c ~ ll, family = binomial, data = x[[i]])

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-0.6124  -0.3589  -0.3589  -0.3318   2.7025  

Coefficients:
              Estimate Std. Error  z value Pr(>|z|)    
(Intercept)  -1.879244   0.001484 -1265.91   <2e-16 ***
llCSS        -0.871920   0.004325  -201.59   <2e-16 ***
llCs         -0.992666   0.003437  -288.79   <2e-16 ***
llFml        -0.762405   0.009832   -77.54   <2e-16 ***
llGo         -0.743766   0.006095  -122.02   <2e-16 ***
llJS         -1.165122   0.001908  -610.58   <2e-16 ***
llKotlin     -1.746322   0.011520  -151.60   <2e-16 ***
llMarkdown   -1.530686   0.003360  -455.63   <2e-16 ***
llObjectiveC -0.666082   0.006866   -97.02   <2e-16 ***
llOther      -0.830693   0.001566  -530.62   <2e-16 ***
llPY         -0.974707   0.003204  -304.20   <2e-16 ***
llR          -0.489500   0.008002   -61.17   <2e-16 ***
llRust       -0.951967   0.008896  -107.01   <2e-16 ***
llScala      -1

### 3. Are binary blobs copied more?

In [32]:
for(i in 2:3) {
    print(names(x)[i])
    test=t.test(x[[i]][x[[i]]$b,"c"],x[[i]][!x[[i]]$b,"c"])
    print(test)
}

[1] "1y"

	Welch Two Sample t-test

data:  x[[i]][x[[i]]$b, "c"] and x[[i]][!x[[i]]$b, "c"]
t = 351.37, df = 36289558, p-value < 2.2e-16
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.02065332 0.02088503
sample estimates:
 mean of x  mean of y 
0.07759141 0.05682223 

[1] "2y"

	Welch Two Sample t-test

data:  x[[i]][x[[i]]$b, "c"] and x[[i]][!x[[i]]$b, "c"]
t = 332.06, df = 26998008, p-value < 2.2e-16
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.02477441 0.02506860
sample estimates:
 mean of x  mean of y 
0.09530248 0.07038098 



### 4. Are copied blobs smaller?

In [68]:
for(i in 2:3) {
    print(names(x)[i])
    print(t.test(log(x[[i]][x[[i]]$c,"s"]),log(x[[i]][!x[[i]]$c,"s"])))
}

[1] "1y"

	Welch Two Sample t-test

data:  log(x[[i]][x[[i]]$c, "s"]) and log(x[[i]][!x[[i]]$c, "s"])
t = -222.22, df = 5528798, p-value < 2.2e-16
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.1731598 -0.1701320
sample estimates:
mean of x mean of y 
 7.167717  7.339363 

[1] "2y"

	Welch Two Sample t-test

data:  log(x[[i]][x[[i]]$c, "s"]) and log(x[[i]][!x[[i]]$c, "s"])
t = -199.3, df = 5348321, p-value < 2.2e-16
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.1608797 -0.1577463
sample estimates:
mean of x mean of y 
 7.142075  7.301388 



#### 4.1. Is this the case for all languages?

In [72]:
sv=0.005
sl=list()
for(i in 2:3){
    sl[[i]]=list()
    print(names(x)[i])
    for(l in ty){ 
        test=t.test(log(x[[i]][x[[i]]$ll==l&x[[i]]$c,"s"]),log(x[[i]][x[[i]]$ll==l&!x[[i]]$c,"s"]))
        if(test$p.value<sv){
            print(c(l,sprintf("t: %f",test$statistic)))
            sl[[i]]=append(sl[[i]],l)
        }else{
            print(c(l,"Not significant!"))
        }
    }
}

[1] "1y"
[1] "C"             "t: 216.836513"
[1] "CSS"          "t: -3.504343"
[1] "Cs"               "Not significant!"
[1] "Fml"              "Not significant!"
[1] "Go"           "t: 24.338372"
[1] "JS"            "t: -62.877606"
[1] "Kotlin"        "t: -14.695897"
[1] "Markdown"      "t: -59.064043"
[1] "ObjectiveC"  "t: 7.621318"
[1] "Other"          "t: -411.342656"
[1] "PY"           "t: 21.424793"
[1] "R"            "t: -7.713302"
[1] "Rust"         "t: -7.192477"
[1] "Scala"        "t: 14.646112"
[1] "Sql"           "t: -12.622829"
[1] "Swift"        "t: -5.014679"
[1] "TypeScript"    "t: -38.828616"
[1] "ipy"          "t: -8.096101"
[1] "java"          "t: 138.465030"
[1] "php"          "t: 39.226481"
[1] "pl"          "t: 9.605817"
[1] "rb"            "t: -11.782498"
[1] "2y"
[1] "C"             "t: 195.919272"
[1] "CSS"           "t: -11.883050"
[1] "Cs"           "t: 12.552948"
[1] "Fml"              "Not significant!"
[1] "Go"           "t: 15.571693"
[1] "JS"            

#### 4.2. How are these languages 

Are older blobs more likely to be copied?

In [76]:
for(i in 1:3) {
    print(names(x)[i])
    print(t.test(x[[i]][x[[i]]$c==1,"t"],x[[i]][x[[i]]$c==0,"t"]))
}

[1] "All"

	Welch Two Sample t-test

data:  x[[i]][x[[i]]$c == 1, "t"] and x[[i]][x[[i]]$c == 0, "t"]
t = -1071, df = 6316731, p-value < 2.2e-16
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -69500652 -69246732
sample estimates:
 mean of x  mean of y 
1463191531 1532565223 

[1] "1y"

	Welch Two Sample t-test

data:  x[[i]][x[[i]]$c == 1, "t"] and x[[i]][x[[i]]$c == 0, "t"]
t = -882.09, df = 5199165, p-value < 2.2e-16
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -55120103 -54875696
sample estimates:
 mean of x  mean of y 
1466230434 1521228333 

[1] "2y"

	Welch Two Sample t-test

data:  x[[i]][x[[i]]$c == 1, "t"] and x[[i]][x[[i]]$c == 0, "t"]
t = -789.95, df = 5005710, p-value < 2.2e-16
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -48927504 -48685314
sample estimates:
 mean of x  mean of y 
1445911059 1494717469 



In [121]:
for(i in 2:3) {
    print(names(x)[i]); 
    print(summary(glm(x[[i]]$c~x[[i]]$t+log(x[[i]]$s))))
}

[1] "1y"

Call:
glm(formula = x[[i]]$c ~ x[[i]]$t + log(x[[i]]$s))

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-0.54951  -0.06806  -0.04733  -0.03328   1.00240  

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    5.515e-01  3.992e-04    1382   <2e-16 ***
x[[i]]$t      -3.120e-10  2.543e-13   -1227   <2e-16 ***
log(x[[i]]$s) -2.860e-03  1.482e-05    -193   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for gaussian family taken to be 0.05264742)

    Null deviance: 4639323  on 86565076  degrees of freedom
Residual deviance: 4557428  on 86565074  degrees of freedom
AIC: -9198488

Number of Fisher Scoring iterations: 2

[1] "2y"

Call:
glm(formula = x[[i]]$c ~ x[[i]]$t + log(x[[i]]$s))

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-0.60902  -0.08085  -0.05983  -0.04513   1.00220  

Coefficients:
                Estimate Std. Error t value Pr(>|t

Does language affect the copied vs not copied time(are older blobs more likely to be copied?)?

In [114]:
for(i in 1:3){
    print(names(x)[i])
    ty=names(table(x[[i]]$l)[table(x[[i]]$l)>100000])
    for(l in ty){ 
        test=t.test(x[[i]][x[[i]]$l==l&x[[i]]$c==1,"t"],x[[i]][x[[i]]$l==l&x[[i]]$c==0,"t"])
        print(c(l,test$statistic,test$p.value));
    }
}

[1] "All"
                                      t                     
                "C" "-413.773065856593"                 "0" 
                                      t                     
              "CSS" "-80.0303948416811"                 "0" 
                                      t                     
               "Cs" "-120.833948190011"                 "0" 
                                      t                     
             "Dart" "-47.4617598463773"                 "0" 
                                      t                     
              "Fml" "-78.4351007017095"                 "0" 
                                      t                     
               "Go" "-46.1543577917912"                 "0" 
                                      t                     
               "JS" "-250.925492207675"                 "0" 
                                            t                        
              "Kotlin"    "-26.2579557404419" "7.5368102042458e-14

In [133]:
for(i in 2:3) {
    print(names(x)[i]); 
    ty=names(table(x[[i]]$l)[table(x[[i]]$l)>100000])
    x[[i]]$ll=as.factor(ifelse(x[[i]]$l %in% ty, as.character(x[[i]]$l), "Other"))
    print(summary(glm(x[[i]]$c~x[[i]]$t+log(x[[i]]$s)+x[[i]]$ll), family=binomial))
}

[1] "1y"

Call:
glm(formula = x[[i]]$c ~ x[[i]]$t + log(x[[i]]$s) + x[[i]]$ll)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-0.57999  -0.06784  -0.04623  -0.03142   1.01871  

Coefficients:
                      Estimate Std. Error  t value Pr(>|t|)    
(Intercept)          5.789e-01  4.101e-04  1411.59   <2e-16 ***
x[[i]]$t            -2.894e-10  2.617e-13 -1105.58   <2e-16 ***
log(x[[i]]$s)       -3.408e-03  1.503e-05  -226.78   <2e-16 ***
x[[i]]$llCSS        -5.380e-02  2.498e-04  -215.33   <2e-16 ***
x[[i]]$llCs         -6.430e-02  1.979e-04  -324.90   <2e-16 ***
x[[i]]$llDart       -5.192e-02  6.422e-04   -80.84   <2e-16 ***
x[[i]]$llFml        -5.965e-02  5.664e-04  -105.32   <2e-16 ***
x[[i]]$llGo         -4.022e-02  3.598e-04  -111.79   <2e-16 ***
x[[i]]$llJS         -6.207e-02  1.310e-04  -473.72   <2e-16 ***
x[[i]]$llKotlin     -7.660e-02  4.327e-04  -177.00   <2e-16 ***
x[[i]]$llLua        -8.647e-03  6.779e-04   -12.76   <2e-16 ***
x[[i]]$llMarkd

In [134]:
for(i in 2:3) {
    print(names(x)[i]); 
    print(summary(glm(x[[i]]$c~log(x[[i]]$s)+x[[i]]$ll), family=binomial))
}

[1] "1y"

Call:
glm(formula = x[[i]]$c ~ log(x[[i]]$s) + x[[i]]$ll)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-0.19290  -0.05932  -0.05170  -0.04555   1.00742  

Coefficients:
                      Estimate Std. Error t value Pr(>|t|)    
(Intercept)          1.624e-01  1.632e-04  995.14   <2e-16 ***
log(x[[i]]$s)       -3.944e-03  1.513e-05 -260.71   <2e-16 ***
x[[i]]$llCSS        -7.354e-02  2.510e-04 -293.03   <2e-16 ***
x[[i]]$llCs         -8.161e-02  1.987e-04 -410.75   <2e-16 ***
x[[i]]$llDart       -8.800e-02  6.459e-04 -136.24   <2e-16 ***
x[[i]]$llFml        -6.598e-02  5.703e-04 -115.69   <2e-16 ***
x[[i]]$llGo         -6.589e-02  3.615e-04 -182.24   <2e-16 ***
x[[i]]$llJS         -8.872e-02  1.297e-04 -684.00   <2e-16 ***
x[[i]]$llKotlin     -1.102e-01  4.347e-04 -253.47   <2e-16 ***
x[[i]]$llLua        -1.949e-02  6.826e-04  -28.55   <2e-16 ***
x[[i]]$llMarkdown   -1.035e-01  1.690e-04 -612.55   <2e-16 ***
x[[i]]$llObjectiveC -6.073e-02  4.179

In [135]:
for(i in 2:3) {
    print(names(x)[i]); 
    print(summary(glm(x[[i]]$c~x[[i]]$t+x[[i]]$ll), family=binomial))
}

[1] "1y"

Call:
glm(formula = x[[i]]$c ~ x[[i]]$t + x[[i]]$ll)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-0.57415  -0.06734  -0.04579  -0.03143   1.00234  

Coefficients:
                      Estimate Std. Error  t value Pr(>|t|)    
(Intercept)          5.558e-01  3.974e-04  1398.65   <2e-16 ***
x[[i]]$t            -2.913e-10  2.617e-13 -1113.13   <2e-16 ***
x[[i]]$llCSS        -5.273e-02  2.499e-04  -211.03   <2e-16 ***
x[[i]]$llCs         -6.186e-02  1.977e-04  -312.96   <2e-16 ***
x[[i]]$llDart       -4.965e-02  6.423e-04   -77.30   <2e-16 ***
x[[i]]$llFml        -5.960e-02  5.665e-04  -105.20   <2e-16 ***
x[[i]]$llGo         -3.911e-02  3.598e-04  -108.70   <2e-16 ***
x[[i]]$llJS         -6.043e-02  1.309e-04  -461.72   <2e-16 ***
x[[i]]$llKotlin     -7.322e-02  4.326e-04  -169.24   <2e-16 ***
x[[i]]$llLua        -8.428e-03  6.781e-04   -12.43   <2e-16 ***
x[[i]]$llMarkdown   -7.583e-02  1.689e-04  -448.94   <2e-16 ***
x[[i]]$llObjectiveC -5.708e-02